# Sección C: Evaluando BOPS en Home & Kitchen

## Introducción

Home & Kitchen es una gran cadena retailer con 84 tiendas en Norte América, 67 de ellas en Estados Unidos y 17 en Canadá. Ha tenido desde el inicio presencia de tiendas físicas y en el 2007 introdujo su tienda online. Ahora cada división, física y online opera como unidad rentable. Después de un fuerte crecimiento de la división online, sus ventas se han estabilizado.

A fin de impulsar las ventas online, en el 2010 se lanzó la iniciativa BOPS (Compre online y recójalo en la tienda, por sus siglas en inglés). Aparte del despacho clásico, los clientes pueden recoger sus compras. Sin embargo BOPS es operacionalmente complejo, y debido a la rivalidad interna entre las divisiones físicas y online los primeros han tenido resistencia al programa BOPS, que sólo ha sido lanzado en Estados Unidos.

En abril del 2012 se tuvieron resultados de las ventas, donde una aparente baja de ventas en ambas divisiones pone en riesgo la continuidad de BOPS. Se pide analizar el caso y recomendar si prosigue la expansión a Canadá y cuantos millones de dólares se ganaron o perdieron con BOPS. 

### Análisis Exploratorio

Iniciaremos cargando las bibliotecas de funciones y los datos que fueron descargados en la carpeta de trabajo. Cada bloque de este notebook se complementará con comentarios para guiar al lector en el procedimiento utilizado.

In [1]:
# Step 0. Cargar bibliotecas y funciones ·······················#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Step 1. Cargar los datos del archivo CSV ·····················#
# 1.1 Cada división tiene su propio archivo, abriremos el canal físico
dataset_bm = pd.read_csv('bops_bm.csv')
# 1.2 Ahora abrimos el canal online
dataset_ol = pd.read_csv('bops_online.csv')

In [3]:
# Step 2. Realizar un análisis exploratorio de datos ···········#
# 2.1. Primero demos un vistazo de las cinco primeras observaciones del dataset del canal físico
dataset_bm.head(5)

,id (store),year,month,week,usa,after,sales,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1.0,2011.0,4.0,16.0,0.0,0.0,"118,691",NaN,NaN,NaN,NaN
1,1.0,2011.0,4.0,17.0,0.0,0.0,"113,804",NaN,NaN,NaN,NaN
2,1.0,2011.0,4.0,18.0,0.0,0.0,"172,104",NaN,NaN,NaN,NaN
3,1.0,2011.0,5.0,19.0,0.0,0.0,"105,591",NaN,NaN,NaN,NaN
4,1.0,2011.0,5.0,20.0,0.0,0.0,"94,884",NaN,NaN,NaN,NaN


In [4]:
# 2.2. Ahora veamos el dataset del canal online
dataset_ol.head(5)

,id (DMA),year,month,week,after,close,sales,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,1,2011,4,17,0,1,"18,564",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2011,4,18,0,1,"30,883",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2011,5,19,0,1,"37,425",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2011,5,20,0,1,"32,563",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2011,5,21,0,1,"35,773",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


De acuerdo al diccionario de datos podiamos esperar 7 columnas en el dataset del canal físico, y 7 columnas en el dataset del canal online. Pasemos a observar el tipo de datos.

In [10]:
# 2.3. Revisemos el tipo de dato de cada variable del canal físico
dataset_bm.dtypes

id (store)    float64
year          float64
month         float64
week          float64
usa           float64
after         float64
 sales         object
dtype: object

In [11]:
# 2.4. Revisemos el tipo de dato de cada variable del canal online
dataset_ol.dtypes

id (DMA)     int64
year         int64
month        int64
week         int64
after        int64
close        int64
 sales      object
dtype: object

Podemos observar que las ventas no están en el formato númerico que esperamos en ambos datasets. Asimismo, sería práctico convertir el año, mes y semana en un formato fecha a fín de utilizarlo como una variable de serie de tiempo. Pasemos a hacer dichas conversiones.

In [20]:
# 2.5 Seleccionamos las primeras 7 columnas en ambos datasets
dataset_bm = dataset_bm.iloc[:,:7]
dataset_ol = dataset_ol.iloc[:,:7]
# 2.6 Convertimos los nombres de los campos de ventas en numérico 
dataset_bm.rename(columns = {dataset_bm.columns[6]:'sales'}, inplace = True)
dataset_ol.rename(columns = {dataset_ol.columns[6]:'sales'}, inplace = True)
# 2.7 Convertimos los campos ventas en numéricos
dataset_bm['sales'] = dataset_bm['sales'].str.replace(',','').astype('float64')
dataset_ol['sales'] = dataset_ol['sales'].str.replace(',','').astype('float64')
# 2.8 Convertimos los campos año, mes y semana en fecha e indexamos

In [21]:
dataset_bm.describe()

,id (store),year,month,week,usa,after,sales
count,4536.000000,4536.000000,4536.000000,4536.000000,4536.000000,4536.000000,4536.000000
mean,84.738095,2011.296296,6.314815,26.796296,0.797619,0.500000,63873.317019
std,49.434118,0.456674,3.479549,15.228823,0.401819,0.500055,36531.546280
min,1.000000,2011.000000,1.000000,1.000000,0.000000,0.000000,2915.000000
25%,42.500000,2011.000000,3.000000,14.000000,1.000000,0.000000,37465.500000
50%,84.000000,2011.000000,6.000000,26.500000,1.000000,0.500000,58165.000000
75%,127.500000,2012.000000,9.000000,40.000000,1.000000,1.000000,84089.750000
max,169.000000,2012.000000,12.000000,53.000000,1.000000,1.000000,286827.000000


Podemos observar que hay 4536 observaciones, 

In [8]:
dataset_bm.dtypes

id (store)    float64
year          float64
month         float64
week          float64
usa           float64
after         float64
 sales         object
dtype: object

In [9]:
dataset_ol.dtypes

id (DMA)     int64
year         int64
month        int64
week         int64
after        int64
close        int64
 sales      object
dtype: object